In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input, Bidirectional
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import math
from datetime import datetime
from tqdm import tqdm
print(f'Current Step #01 [now = {datetime.now()}')


2025-06-20 22:41:50.594050: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Current Step #01 [now = 2025-06-20 22:41:53.724955


In [2]:
results = {}

In [3]:
def load_metrics(random_state, version, db_file_path):
    query = f'select rounds, matched_cnts from rndforest where random_state={random_state} and version=\\"{version}\\"'
    metrics = !echo {query} | sqlite3 {db_file_path}
    metrics = metrics[0].split('|')
    return metrics[0], metrics[1]


def load_data(version="T_01_10", db_file_path='../db/metrics.db', random_state=113789):
    rounds_str, matched_cnts_str = load_metrics(random_state=random_state, db_file_path=db_file_path, version=version)
    rounds = []
    matched_cnts = []
    for i in rounds_str.split(','):
        rounds.append(int(i))
    for i in matched_cnts_str.split(','):
        matched_cnts.append(int(i))
    rounds.reverse()
    matched_cnts.reverse()
    df = pd.DataFrame(zip(rounds, matched_cnts), columns=['rounds', 'matched_cnts'])
    return df


def create_sequences(data, seq_length):
    xs = []
    ys = []
    for i in range(len(data) - seq_length - 1):
        x = data[i:(i + seq_length)]
        y = data[i + seq_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

In [4]:
time_steps = 10
data_dim = 1
num_samples = 100
X = np.random.rand(num_samples, time_steps, data_dim)
print(len(X))

100


```python
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dropout,  Dense
from tensorflow.keras.regularizers import l1_l2, l2

total_words = 478
max_sequence_len = 90
model = Sequential()
Layer1 = model.add(Embedding(total_words, 64, input_length=max_sequence_len-1))
Layer2 = model.add(Bidirectional(LSTM(20, return_sequences=True)))
Layer3 = model.add(Dropout(.03))
Layer4 = model.add(LSTM(20))
Layer5 = model.add(Dense(total_words, 
    kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4),
    bias_regularizer=l2(1e-4),
    activity_regularizer=l2(1e-5)))
          # A Dense Layer including regularizers
Layer6 = model.add(Dense(total_words, activation = 'softmax'))
          
# Pick an optimizer
          
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()
```

In [5]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# Example data
data = np.array([[1], [2], [3], [4], [5]])

# Initialize and fit-transform the scaler
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)

print("Scaled Data:")
print(scaled_data)

new_scaled = np.array([0.33, 0.5, 0.83456, 0.845])
new_scaled = new_scaled.reshape(-1, 1)

# Restore the original data using inverse_transform
restored_data = scaler.inverse_transform(scaled_data)
restored_data1 = scaler.inverse_transform(new_scaled)
print("\nRestored Data:")
print(restored_data)
print(restored_data1)


Scaled Data:
[[0.  ]
 [0.25]
 [0.5 ]
 [0.75]
 [1.  ]]

Restored Data:
[[1.]
 [2.]
 [3.]
 [4.]
 [5.]]
[[2.32   ]
 [3.     ]
 [4.33824]
 [4.38   ]]


In [6]:
from enum import Enum


class PrivateScaler:
    def __init__(self, divider=6.0):
        """" __init___ """
        self._divider = divider
        self.p_max = 1
        self.p_min = 0

    def fit_transform(self, data):
        """ fit_transform """
        self.p_max = data.max() + 1
        self._divider = self.p_max
        scaled_data = np.array([[round(float(i[0])/self._divider, 9)] for i in data])
        return scaled_data

    def inverse_transform(self, data):
        """ inverse_transform """
        self.p_min = data.min()
        if len(data) > 1:
            restored = np.array([[((i[0] - self.p_min)*self._divider)] for i in data])
            r_max = restored.max()
            while r_max < 0.1:
                restored *= 10
                r_max = restored.max()
        else:
            restored = np.array([[((i[0])*self._divider)] for i in data])
        return restored


class ScalerType(Enum):
    PRIVATE=0,
    STANDARD=1,
    MIN_MAX=2


scaler_cls = {
    ScalerType.PRIVATE: PrivateScaler,
    ScalerType.STANDARD: StandardScaler,
    ScalerType.MIN_MAX: MinMaxScaler # feature_range=(0,1)
    }


class DataScaling():
    def __init__(self, scaler_type: ScalerType, *args: list):
        self._scaler_type = scaler_type
        self._scaler_cls = scaler_cls[scaler_type](*args)

    def fit_transform(self, data):
        return self._scaler_cls.fit_transform(data)

    def inverse_transform(self, datas: list):
        inversed_data = []
        if self._scaler_type == ScalerType.PRIVATE:
            for data in datas:
                if data[1] == True:
                    inversed = self._scaler_cls.inverse_transform(data[0].reshape(-1, 1))
                else:
                    inversed = self._scaler_cls.inverse_transform(data[0])
                inversed_data.append(inversed)
        else:
            cnt = 1
            for data in datas:
                print(f'cnt = {cnt}')
                cnt += 1
                if data[1] == True:
                    inversed = self._scaler_cls.inverse_transform(data[0].reshape(-1, 1))
                else:
                    inversed = self._scaler_cls.inverse_transform(data[0])
                inversed_data.append(inversed)
        return inversed_data


### 최종 결과 입니다.
def analyze_v1(random_state,
               version,
               db_file_path,
               predict_round=7,
               predict_scale=10000000,
               scaler_type=ScalerType.PRIVATE,
               layer_count=3,
               activation='sigmoid',
               units=50,
               draw_graph=True,
               verbose=0):
    import numpy as np
    import pandas as pd
    from sklearn.preprocessing import MinMaxScaler
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import LSTM, Dense

    # 1. Prepare the Data
    # Create sample time series data
    df = load_data(version=version, db_file_path=db_file_path, random_state=random_state)
    if verbose > 0:
        print('df', df)
    if verbose > 0:
        print(f'Current Step #02 [now = {datetime.now()}]')

    data = df['matched_cnts'].values.reshape(-1, 1)

    # Normalize the data
    scaler = DataScaling(scaler_type)
    scaled_data = scaler.fit_transform(data)
    if verbose > 0:
        print(f'Current Step #04 [now = {datetime.now()}]')

    # Define sequence length (timesteps)
    sequence_length = 10

    # Create sequences for training
    X, y = [], []
    for i in range(len(scaled_data) - sequence_length):
        X.append(scaled_data[i:i + sequence_length, 0])
        y.append(scaled_data[i + sequence_length, 0])
    X, y = np.array(X), np.array(y)
    if verbose > 0:
        print(f'Current Step #06 [now = {datetime.now()}]')

    # Reshape X for LSTM input (samples, timesteps, features)
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))
    if verbose > 0:
        print(f'Current Step #08 [now = {datetime.now()}]')

    # Split data into training and testing sets
    # train_size = int(len(X) * 0.8)
    train_size = int(len(X) * 0.9)

    X_train, X_test, X_last = X[:train_size], X[train_size:], np.array([[y[-1:]]])
    y_train, y_test = y[:train_size], y[train_size:]
    if verbose > 0:
        print(f'Current Step #10 [now = {datetime.now()}]')

    tf.random.set_seed(random_state)

    # 2. Build the LSTM Model
    model = Sequential()
    model.add(Input(shape=(X_train.shape[1], 1)))
    # model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
    for i in range(layer_count):
        model.add(LSTM(units=units, return_sequences=True, activation=activation))
    model.add(LSTM(units=units, activation=activation))
    model.add(Dense(units=1)) # Output layer for predicting a single value
    if verbose > 0:
        print(f'Current Step #12 [now = {datetime.now()}]')

    # 3. Compile and Train the Model
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(X_train, y_train, epochs=50, batch_size=1, verbose=0) # verbose=0 for silent training

    if verbose > 0:
        print(f'Current Step #14 [now = {datetime.now()}]')

    # 4. Make Predictions
    train_predict = model.predict(X_train, verbose=verbose)
    test_predict = model.predict(X_test, verbose=verbose)
    last_predict = model.predict(X_last, verbose=verbose)
    if verbose > 0:
        print(f'Current Step #16 [now = {datetime.now()}]')

    # Inverse transform predictions to original scale
    scaled_data1 = scaled_data
    wanted_datas = [(train_predict, False),
                    (test_predict, False),
                    (last_predict, True),
                    (y_train, True),
                    (y_test, True),
                    (scaled_data, False)]
    inversed_datas = scaler.inverse_transform(wanted_datas)
    train_predict = inversed_datas[0]
    test_predict = inversed_datas[1]
    last_predict = inversed_datas[2]
    y_train_original = inversed_datas[3]
    y_test_original = inversed_datas[4]
    scaled_data = inversed_datas[5]

    if verbose > 0:
        print(f'Current Step #18 [now = {datetime.now()}]')

    # 5. Evaluate the Model (Optional, but recommended)
    from sklearn.metrics import mean_squared_error
    train_rmse = np.sqrt(mean_squared_error(y_train_original, train_predict))
    test_rmse = np.sqrt(mean_squared_error(y_test_original, test_predict))

    if verbose > 0:
        print(f'Current Step #20 [now = {datetime.now()}]')
        print(f"Train RMSE: {train_rmse}")
        print(f"Test RMSE: {test_rmse}")
        print(f'random_state = {random_state}')
        print(f'last_prediction = {last_predict}')
        print(f'data_max_val = {scaled_data.max()}')
    # You can also visualize the results
    if draw_graph:
        import matplotlib.pyplot as plt
        plt.figure(figsize=(12, 6))
        plt.plot(scaled_data1, label='Original Data')
        plt.plot(np.arange(sequence_length,
                           sequence_length + len(train_predict)),
                 train_predict,
                 label='Train Predictions')
        plt.plot(np.arange(sequence_length + len(train_predict),
                           sequence_length + len(train_predict) + len(test_predict)),
                 test_predict,
                 label='Test Predictions')
        plt.plot(np.arange(sequence_length + len(train_predict) + len(test_predict),
                       sequence_length + len(train_predict) + len(test_predict) + len(last_predict)),
                 last_predict,
                 label='Last Predictions')
        plt.xlabel(f'{random_state}\'s Time Step')
        plt.ylabel('Value')
        plt.legend()
        plt.show()
    return scaled_data.max(), last_predict, random_state


print(f'Current work : {datetime.now()}')

Current work : 2025-06-20 22:41:53.787800


In [9]:
version = "T_01_12"

In [ ]:
db_file_path = '../db/metrics.db'
random_state = 113708
predict_scale = 10000000
predict_round = 8
scaler_type=ScalerType.PRIVATE
# layer information
layer_count = 3
activation = 'sigmoid'
units = 50

analyze_v1(random_state=random_state,
           version=version,
           predict_round=predict_round,
           db_file_path=db_file_path,
           scaler_type=scaler_type,
           layer_count=layer_count,
           activation=activation,
           units=units,
           verbose=0)

In [ ]:
db_file_path = '../db/metrics.db'
random_state = 113799
# predict_scale = 10000000
predict_scale = 1000000
predict_round = 7
scaler_type=ScalerType.PRIVATE
# layer information
layer_count = 3
activation = 'sigmoid'
units = 50
analyze_v1(random_state=random_state, version=version, db_file_path=db_file_path, verbose=0)

In [ ]:
### 최종 결과 입니다. (113789)
db_file_path = '../db/metrics.db'
random_state = 113789
predict_scale = 10000000
predict_round = 7
scaler_type=ScalerType.PRIVATE
# layer information
layer_count = 3
activation = 'sigmoid'
units = 50
analyze_v1(random_state=random_state,
           version=version,
           predict_round=predict_round,
           db_file_path=db_file_path,
           scaler_type=scaler_type,
           layer_count=layer_count,
           activation=activation,
           units=units,
           verbose=0)

In [ ]:
### 최종 결과 입니다. (113789)
db_file_path = '../db/metrics.db'
random_state = 113700
predict_scale = 10000000
predict_round = 7
layer_count = 3
activation = 'sigmoid'
units = 50
scaler_type=ScalerType.PRIVATE
analyze_v1(random_state=random_state,
           version=version,
           predict_round=predict_round,
           db_file_path=db_file_path,
           scaler_type=scaler_type,
           layer_count=layer_count,
           activation=activation,
           units=units,
           verbose=0)

In [ ]:
db_file_path = '../db/metrics.db'
random_state = 113705
# predict_scale = 10000000
predict_scale = 1000000
predict_round = 7
layer_count = 3
activation = 'sigmoid'
units = 50
scaler_type=ScalerType.PRIVATE
analyze_v1(random_state=random_state,
           version=version,
           predict_round=predict_round,
           db_file_path=db_file_path,
           scaler_type=scaler_type,
           layer_count=layer_count,
           activation=activation,
           units=units,
           verbose=0)

In [8]:
"""
version TEXT, random_state INTEGER, possibility FLOAT, max INTEGER
"""
def insert_possi(version, max, possi, random_state):
    table_name='rndforest_possi'
    db_file_path='../db/metrics.db'
    sql = f'insert into {table_name} (version, random_state, possibility, max) values(\\"{version}\\", {random_state}, {possi}, {max});'
    !echo "{sql}" | sqlite3 {db_file_path}

print(f'Current Step : [now {datetime.now()}]')

Current Step : [now 2025-06-20 22:43:02.880812]


In [15]:
def get_possibility(random_states, db_file_path, layer_count, activation, units, scaler_type):
    random_state_begin = 3700
    # predict_scale = 10000000
    predict_scale = 1000000
    predict_round = 7
    results = []
    for random_state in tqdm(random_states):
        scaled_data_max, last_predict, random_state = analyze_v1(random_state=random_state,
                                                                 version=version,
                                                                 predict_round=predict_round,
                                                                 db_file_path=db_file_path,
                                                                 scaler_type=scaler_type,
                                                                 layer_count=layer_count,
                                                                 activation=activation,
                                                                 units=units,
                                                                 draw_graph=False,
                                                                 verbose=0)
        results.append((scaled_data_max, last_predict, random_state))
        if last_predict > 1.7:
            print(scaled_data_max, last_predict, random_state)
    return results

In [10]:
results=[]
#113780에서 5씩 증가

db_file_path = '../db/metrics.db'
layer_count = 3
activation = 'sigmoid'
units = 50
gap = 5
scaler_type = ScalerType.PRIVATE
random_states = range(random_state_begin, random_state_begin+gap)
results = get_possibility(random_states=random_states,
                          db_file_path=db_file_path,
                          layer_count=layer_count,
                          activation=activation,
                          units=units,
                          scaler_type=scaler_type)


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [11:18<00:00, 135.68s/it]


In [16]:
results=[]
#113780에서 5씩 증가

db_file_path = '../db/metrics.db'
layer_count = 3
activation = 'sigmoid'
units = 50
scaler_type=ScalerType.PRIVATE
rows = "\
('74692', '3705', 10)\
('74702', '3715', 10)\
('74704', '3717', 10)\
('74712', '3725', 10)\
('74721', '3734', 10)\
('74740', '3753', 10)\
('74743', '3756', 10)\
('74748', '3761', 10)\
('74757', '3770', 10)\
('74775', '3788', 10)\
('74777', '3790', 10)\
('74779', '3792', 10)\
('74698', '3711', 11)\
('74715', '3728', 11)\
('74716', '3729', 11)\
('74730', '3743', 11)\
('74746', '3759', 11)\
('74778', '3791', 11)\
('74781', '3794', 11)\
('74722', '3735', 12)\
('74766', '3779', 12)\
('74767', '3780', 12)\
('74782', '3795', 12)\
('74696', '3709', 13)\
('74699', '3712', 13)\
('74711', '3724', 13)\
('74723', '3736', 13)\
('74729', '3742', 13)\
('74720', '3733', 14)\
"
# ('74689', '3702', 10)\

random_states = []
data_rows_str = rows.replace("(","")
data_rows = data_rows_str.split(")")
for row in data_rows:
    if len(row) > 0:
        row = row.replace("\'","")
        cols = row.split(", ")
        if len(cols) > 0:
            random_states.append(int(cols[1]))
print(random_states)
results = get_possibility(random_states=random_states,
                          db_file_path=db_file_path,
                          layer_count=layer_count,
                          activation=activation,
                          units=units,
                          scaler_type=scaler_type)

[3705, 3715, 3717, 3725, 3734, 3753, 3756, 3761, 3770, 3788, 3790, 3792, 3711, 3728, 3729, 3743, 3759, 3791, 3794, 3735, 3779, 3780, 3795, 3709, 3712, 3724, 3736, 3742, 3733]


100%|████████████████████████████████████████████████████████████████████████████████| 29/29 [1:09:35<00:00, 143.99s/it]


In [17]:
for result in results:
    print(result)
    insert_possi(version=version, max=result[0], possi=result[1][0][0], random_state=result[2])

(np.float64(4.0), array([[0.56042761]]), 3705)
(np.float64(3.0), array([[1.21194208]]), 3715)
(np.float64(4.0), array([[0.40450912]]), 3717)
(np.float64(3.0), array([[1.46030033]]), 3725)
(np.float64(3.0), array([[0.95082545]]), 3734)
(np.float64(4.0), array([[0.8309944]]), 3753)
(np.float64(3.0), array([[1.22648871]]), 3756)
(np.float64(4.0), array([[0.7854557]]), 3761)
(np.float64(3.0), array([[1.08949208]]), 3770)
(np.float64(3.0), array([[0.14246422]]), 3788)
(np.float64(3.0), array([[0.74340159]]), 3790)
(np.float64(3.0), array([[0.53213376]]), 3792)
(np.float64(4.0), array([[0.78057639]]), 3711)
(np.float64(4.0), array([[0.38172998]]), 3728)
(np.float64(4.0), array([[0.18725734]]), 3729)
(np.float64(3.0), array([[0.4299612]]), 3743)
(np.float64(3.0), array([[1.04214549]]), 3759)
(np.float64(4.0), array([[0.92168175]]), 3791)
(np.float64(3.0), array([[0.29263979]]), 3794)
(np.float64(4.0), array([[0.79913124]]), 3735)
(np.float64(4.0), array([[0.2566186]]), 3779)
(np.float64(3.0),

In [ ]:

actual_numbers = [
    [1175, 3, 4, 6, 8, 32, 42],
    [1174, 8, 11, 14, 17, 36, 39],
    [1173, 1, 5, 18, 20, 30, 35],
    [1172, 7, 9, 24, 40, 42, 44],
    [1171, 3, 6, 7, 11, 12, 17],
    [1170, 3, 13, 28, 34, 38, 42],
    [1169, 5, 12, 24, 26, 39, 42],
    [1168, 9, 21, 24, 30, 33, 37],
    [1167, 8, 23, 31, 35, 39, 40],
]